<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Market-Data/blob/main/Get_Funding_Rate_History_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 取得資金費率歷史紀錄 (Get Funding Rate History)
# GET /openApi/swap/v2/quote/fundingRate
# 不需要 API 金鑰簽名

In [ ]:
# 如果未傳送 startTime 與 endTime，則回傳最新的 limit 筆資料。
# 回傳清單依時間由小到大排序。
# 如果 startTime 與 endTime 間的資料量大於 limit，則回傳 startTime 起 limit 筆資料。

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 基本網址
APIKEY = ""  # 使用者的 API 金鑰，需要填入有效金鑰
SECRETKEY = ""  # 使用者的秘密金鑰，需要填入有效密鑰

In [ ]:
def demo():
    payload = {}  # GET 請求通常不需請求體
    path = '/openApi/swap/v2/quote/fundingRate'  # API 路徑：取得資金費率歷史
    method = "GET"  # HTTP 請求方法

    paramsMap = {
        "symbol": "QNT-USDT",  # 交易對，需包含連字號，例如 QNT-USDT
        "limit": 2  # 限制回傳資料筆數，這裡設定為 2 筆
    }

    paramsStr = parseParam(paramsMap)  # 將參數字典轉成 URL 查詢字串並加上 timestamp
    return send_request(method, path, paramsStr, payload)  # 發送請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):

    # 使用 HMAC-SHA256 演算法產生簽名，payload 是排序後的參數字串
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名方便調試
    return signature  # 回傳簽名

In [ ]:
def send_request(method, path, urlpa, payload):

    # 組合完整 URL，包含簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出完整請求 URL，方便除錯

    headers = {
        'X-BX-APIKEY': APIKEY,  # HTTP 標頭帶入 API 金鑰
    }

    # 發送 HTTP 請求，帶入方法、URL、標頭與請求體
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text  # 回傳 API 回應內容（字串）

In [ ]:
def parseParam(paramsMap):
    sortedKeys = sorted(paramsMap)  # 對參數鍵排序以確保簽名一致

    # 將參數轉成 URL 查詢字串格式，如 key=value&key2=value2
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    if paramsStr != "":
        # 若有其他參數，加入 timestamp 時間戳（毫秒）
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 無其他參數時，只加 timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行示範函式並印出結果